::: {.callout-note appearance="simple"}
### Licensing Notice
Text and media: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)
Code and snippets: [Apache License 2.0](http://www.apache.org/licenses/LICENSE-2.0)
:::

## Motivation and Meaning

**R**etrieval **A**ugmented **G**eneration (RAG) is inference-time context injection: pull **relevant** external data and condition generation on it. This *grounds* responses in authoritative sources while keeping LLMs parametric knowledge (weights) frozen.

#### Motivation
- **Knowledge limited to training data**: Your proprietary/domain-specific data isn't there
- **Fixed knowledge cut-off date**: The model can't answer questions about recent events. Without RAG, they will either refuse to answer or *hallucinate*.

##### Alternatives & Trade-offs
- Why not fine-tune?
    - Fine-tuning excels at teaching *task formats* (SQL generation, JSON output) and *reasoning styles*, not injecting factual knowledge.
    - **Catastrophic forgetting**: Updating knowledge degrades performance on other tasks as model weights get overwritten.
    - **Inefficient**: requires separate fine-tuned checkpoints per domain or use-case or document. It is tricky to learn new knowledge without regressing on old knowledge even with LoRA/QLoRA. 
- Ok, why not just stuff everything in the context window?
    - **Recall Degradation**: While 1M+ token models ace "single-needle" tests, performance drops significantly (to ~60-70%) when retrieving multiple distributed facts
    - **Cost & Latency**: Processing massive contexts is computationally expensive and slow compared to vector search. Retrieval remains necessary for corpora exceeding the window size.
- RAG is a reasonable pattern:
    - When you need fresh, attributable knowledge with minimal model changes and can tolerate added latency.

#### The RAG Pipeline
RAG acts as a filter to inject only *relevant* context. A typical production pipeline looks like this:

0. **Ingestion & Indexing**: Chunk documents, generate embeddings, and upsert into a vector database. *Note: In production, this is a continuous sync pipeline, not a one-time setup.*
1. **Retrieval**: For a user query, search your indexed corpus (vector/keyword) and pull the top‑k relevant chunks, often followed by a *re-ranking* step for precision.
2. **Augmented (prompt)**: Inject selected chunks into the system prompt or user message with appropriate metadata (source citations).
3. **Generation**: The LLM generates an answer conditioned *strictly* on the provided context, minimizing external knowledge leakage.

```{mermaid}
%%| label: fig-mermaid-ragv1
%%| fig-cap: "RAG pipeline flowchart showing ingestion pipeline, query processing, retrieval, augmentation and LLM generation."

flowchart TB
    n2["LLM"] L_n2_n4_0@-- generates grounded answer --> n4["Answer"]
    n3["Document Corpus"] L_n3_n5_0@<-- ingestion pipeline<br>(chunk + embed) --> n5["Hybrid index<br>(inverted keywords<br>+ <br>vector embeddings)<br><br>"]
    n5 L_n5_n6_0@-- "top-k" --> n6["Retrieval &amp; Re-ranking"]
    n6 L_n6_n7_0@-- "top-k re-ranked chunks + citation metadata" --> n7["Prompt Builder"]
    n7 L_n7_n2_0@-- "system prompt (use only given context) + user query + <br>top-k re-ranked chunks &amp; citation metadata" --> n2
    n1["User Query"] L_n1_n8_0@--> n8["Query processing <br>&amp; embedding"]
    n1 L_n1_n7_0@-- user query --> n7
    n8 L_n8_n6_0@-- text + query expansions + embeddings --> n6

    n3@{ shape: docs}
    n5@{ shape: cyl}
    n6@{ shape: rect}
    n7@{ shape: rect}
    n1@{ shape: rect}
    n8@{ shape: rect}

    L_n2_n4_0@{ animation: slow } 
    L_n3_n5_0@{ animation: none } 
    L_n5_n6_0@{ animation: slow } 
    L_n6_n7_0@{ animation: slow } 
    L_n7_n2_0@{ animation: slow } 
    L_n1_n8_0@{ animation: slow } 
    L_n1_n7_0@{ animation: slow } 
    L_n8_n6_0@{ animation: slow }
```